In [1]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
from nuscenes.utils.geometry_utils import points_in_box
from enum import IntEnum
from typing import Tuple
from pyquaternion import Quaternion
import numpy as np
import csv

In [2]:
nusc = NuScenes(version='v1.0-trainval', dataroot='/home/mike/workspaces/ECE579/data', verbose=True)

Loading NuScenes tables for version v1.0-trainval ...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 27.9 seconds.
Reverse indexing ...
Done reverse indexing in 6.8 seconds.


In [4]:
#Grab All Samples available
samples = nusc.sample[:]

#Radar point extraction parameters
keepValidities=[0x00,0x01,0x02,0x03,0x05,0x06,0x07,0x0d,0x0e,0x04,0x08,0x09,0x0a,0x0b,0x0c,0x0f,0x10,0x11]
keepAmbigStates=range(4)

In [6]:
fieldnames = ['sample_annotation_token','category','x','y','z','dyn_prop','id','rcs','vx','vy','vx_comp','vy_comp','is_quality_valid','ambig_state','x_rms','y_rms','invalid_state','pdh0','vx_rms','vy_rms']
with open('labelledRadarPoints.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    samplenum=0
    #Loop over all samples avialable
    for sample in samples:
        samplenum+=1
        print("Sample Number:" + str(samplenum))
        #Obtain all radar sample_data(SD) tokens for this sample
        FC_RadarSD = nusc.get('sample_data', sample['data']['RADAR_FRONT'])
        FL_RadarSD = nusc.get('sample_data', sample['data']['RADAR_FRONT_LEFT'])
        FR_RadarSD = nusc.get('sample_data', sample['data']['RADAR_FRONT_RIGHT'])
        RL_RadarSD = nusc.get('sample_data', sample['data']['RADAR_BACK_LEFT'])
        RR_RadarSD = nusc.get('sample_data', sample['data']['RADAR_BACK_RIGHT'])

        #GRAB Path to point clouds (pc_path), and all annotation (boxes) with this sensor data associated with them.
        #Note that boxes are transformed into sensor frame, so we need a new list of boxes for each sensor. array is only used for cameras
        FC_pc_path, boxesFC, array = nusc.get_sample_data(FC_RadarSD['token'])
        FL_pc_path, boxesFL, array = nusc.get_sample_data(FL_RadarSD['token'])
        FR_pc_path, boxesFR, array = nusc.get_sample_data(FR_RadarSD['token'])
        RL_pc_path, boxesRL, array = nusc.get_sample_data(RL_RadarSD['token'])
        RR_pc_path, boxesRR, array = nusc.get_sample_data(RR_RadarSD['token'])
        boxes = [boxesFC,boxesFL,boxesFR,boxesRL,boxesRR]

        #Obtain each radar point cloud for this sample
        FC_pc = RadarPointCloud.from_file(FC_pc_path,invalid_states=keepValidities,ambig_states=keepAmbigStates)
        FC_points = np.array(FC_pc.points[:3, :])
        FL_pc = RadarPointCloud.from_file(FL_pc_path,invalid_states=keepValidities,ambig_states=keepAmbigStates)
        FL_points = np.array(FL_pc.points[:3, :])
        FR_pc = RadarPointCloud.from_file(FR_pc_path,invalid_states=keepValidities,ambig_states=keepAmbigStates)
        FR_points = np.array(FR_pc.points[:3, :])
        RL_pc = RadarPointCloud.from_file(RL_pc_path,invalid_states=keepValidities,ambig_states=keepAmbigStates)
        RL_points = np.array(RL_pc.points[:3, :])
        RR_pc = RadarPointCloud.from_file(RR_pc_path,invalid_states=keepValidities,ambig_states=keepAmbigStates)
        RR_points = np.array(RR_pc.points[:3, :])
        #Collect all point clouds into one ndarray
        point_clouds = [FC_points,FL_points,FR_points,RL_points,RR_points]
        full_point_clouds = [FC_pc,FL_pc,FR_pc,RL_pc,RR_pc]

        sensor_num=0
        for pc in point_clouds:
            #print("New Sensor")
            this_sensor_boxes = boxes[sensor_num]
            #print(this_sensor_boxes)
            this_sensor_full_pc = full_point_clouds[sensor_num]
            dims, num_points=pc.shape
            for box in this_sensor_boxes:
                box_annotation = nusc.get('sample_annotation',box.token)
                #print("Num Radar Points:" + str(box_annotation['num_radar_pts']))
                if box_annotation['num_radar_pts'] == 0:
                    continue
                #print(box)
                category = box_annotation['category_name']
                point_in_box_result = points_in_box(box, pc)
                for i in range(0, num_points):
                    if point_in_box_result[i]:
                        #print(pc[:3,i])
                        writer.writerow({
                        'sample_annotation_token': box_annotation['token'],
                        'category': category,
                         'x': this_sensor_full_pc.points[0,i],
                         'y': this_sensor_full_pc.points[1,i],
                         'z': this_sensor_full_pc.points[2,i],
                         'dyn_prop': this_sensor_full_pc.points[3,i],
                         'id': this_sensor_full_pc.points[4,i],
                         'rcs': this_sensor_full_pc.points[5,i],
                         'vx': this_sensor_full_pc.points[6,i],
                         'vy': this_sensor_full_pc.points[7,i],
                         'vx_comp': this_sensor_full_pc.points[8,i],
                         'vy_comp': this_sensor_full_pc.points[9,i],
                         'is_quality_valid': this_sensor_full_pc.points[10,i],
                         'ambig_state': this_sensor_full_pc.points[11,i],
                         'x_rms': this_sensor_full_pc.points[12,i],
                         'y_rms': this_sensor_full_pc.points[13,i],
                         'invalid_state': this_sensor_full_pc.points[14,i],
                         'pdh0': this_sensor_full_pc.points[15,i],
                         'vx_rms': this_sensor_full_pc.points[16,i],
                         'vy_rms': this_sensor_full_pc.points[17,i]})
                        #print(this_sensor_full_pc.points[:,i])
            sensor_num+=1

Sample Number:1
Sample Number:2
Sample Number:3
Sample Number:4
Sample Number:5
Sample Number:6
Sample Number:7
Sample Number:8
Sample Number:9
Sample Number:10
Sample Number:11
Sample Number:12
Sample Number:13
Sample Number:14
Sample Number:15
Sample Number:16
Sample Number:17
Sample Number:18
Sample Number:19
Sample Number:20
Sample Number:21
Sample Number:22
Sample Number:23
Sample Number:24
Sample Number:25
Sample Number:26
Sample Number:27
Sample Number:28
Sample Number:29
Sample Number:30
Sample Number:31
Sample Number:32
Sample Number:33
Sample Number:34
Sample Number:35
Sample Number:36
Sample Number:37
Sample Number:38
Sample Number:39
Sample Number:40
Sample Number:41
Sample Number:42
Sample Number:43
Sample Number:44
Sample Number:45
Sample Number:46
Sample Number:47
Sample Number:48
Sample Number:49
Sample Number:50
Sample Number:51
Sample Number:52
Sample Number:53
Sample Number:54
Sample Number:55
Sample Number:56
Sample Number:57
Sample Number:58
Sample Number:59
Sample

Sample Number:476
Sample Number:477
Sample Number:478
Sample Number:479
Sample Number:480
Sample Number:481
Sample Number:482
Sample Number:483
Sample Number:484
Sample Number:485
Sample Number:486
Sample Number:487
Sample Number:488
Sample Number:489
Sample Number:490
Sample Number:491
Sample Number:492
Sample Number:493
Sample Number:494
Sample Number:495
Sample Number:496
Sample Number:497
Sample Number:498
Sample Number:499
Sample Number:500
Sample Number:501
Sample Number:502
Sample Number:503
Sample Number:504
Sample Number:505
Sample Number:506
Sample Number:507
Sample Number:508
Sample Number:509
Sample Number:510
Sample Number:511
Sample Number:512
Sample Number:513
Sample Number:514
Sample Number:515
Sample Number:516
Sample Number:517
Sample Number:518
Sample Number:519
Sample Number:520
Sample Number:521
Sample Number:522
Sample Number:523
Sample Number:524
Sample Number:525
Sample Number:526
Sample Number:527
Sample Number:528
Sample Number:529
Sample Number:530
Sample Num

Sample Number:935
Sample Number:936
Sample Number:937
Sample Number:938
Sample Number:939
Sample Number:940
Sample Number:941
Sample Number:942
Sample Number:943
Sample Number:944
Sample Number:945
Sample Number:946
Sample Number:947
Sample Number:948
Sample Number:949
Sample Number:950
Sample Number:951
Sample Number:952
Sample Number:953
Sample Number:954
Sample Number:955
Sample Number:956
Sample Number:957
Sample Number:958
Sample Number:959
Sample Number:960
Sample Number:961
Sample Number:962
Sample Number:963
Sample Number:964
Sample Number:965
Sample Number:966
Sample Number:967
Sample Number:968
Sample Number:969
Sample Number:970
Sample Number:971
Sample Number:972
Sample Number:973
Sample Number:974
Sample Number:975
Sample Number:976
Sample Number:977
Sample Number:978
Sample Number:979
Sample Number:980
Sample Number:981
Sample Number:982
Sample Number:983
Sample Number:984
Sample Number:985
Sample Number:986
Sample Number:987
Sample Number:988
Sample Number:989
Sample Num

Sample Number:1370
Sample Number:1371
Sample Number:1372
Sample Number:1373
Sample Number:1374
Sample Number:1375
Sample Number:1376
Sample Number:1377
Sample Number:1378
Sample Number:1379
Sample Number:1380
Sample Number:1381
Sample Number:1382
Sample Number:1383
Sample Number:1384
Sample Number:1385
Sample Number:1386
Sample Number:1387
Sample Number:1388
Sample Number:1389
Sample Number:1390
Sample Number:1391
Sample Number:1392
Sample Number:1393
Sample Number:1394
Sample Number:1395
Sample Number:1396
Sample Number:1397
Sample Number:1398
Sample Number:1399
Sample Number:1400
Sample Number:1401
Sample Number:1402
Sample Number:1403
Sample Number:1404
Sample Number:1405
Sample Number:1406
Sample Number:1407
Sample Number:1408
Sample Number:1409
Sample Number:1410
Sample Number:1411
Sample Number:1412
Sample Number:1413
Sample Number:1414
Sample Number:1415
Sample Number:1416
Sample Number:1417
Sample Number:1418
Sample Number:1419
Sample Number:1420
Sample Number:1421
Sample Numbe

Sample Number:1803
Sample Number:1804
Sample Number:1805
Sample Number:1806
Sample Number:1807
Sample Number:1808
Sample Number:1809
Sample Number:1810
Sample Number:1811
Sample Number:1812
Sample Number:1813
Sample Number:1814
Sample Number:1815
Sample Number:1816
Sample Number:1817
Sample Number:1818
Sample Number:1819
Sample Number:1820
Sample Number:1821
Sample Number:1822
Sample Number:1823
Sample Number:1824
Sample Number:1825
Sample Number:1826
Sample Number:1827
Sample Number:1828
Sample Number:1829
Sample Number:1830
Sample Number:1831
Sample Number:1832
Sample Number:1833
Sample Number:1834
Sample Number:1835
Sample Number:1836
Sample Number:1837
Sample Number:1838
Sample Number:1839
Sample Number:1840
Sample Number:1841
Sample Number:1842
Sample Number:1843
Sample Number:1844
Sample Number:1845
Sample Number:1846
Sample Number:1847
Sample Number:1848
Sample Number:1849
Sample Number:1850
Sample Number:1851
Sample Number:1852
Sample Number:1853
Sample Number:1854
Sample Numbe

...

Sample Number:32416
Sample Number:32417
Sample Number:32418
Sample Number:32419
Sample Number:32420
Sample Number:32421
Sample Number:32422
Sample Number:32423
Sample Number:32424
Sample Number:32425
Sample Number:32426
Sample Number:32427
Sample Number:32428
Sample Number:32429
Sample Number:32430
Sample Number:32431
Sample Number:32432
Sample Number:32433
Sample Number:32434
Sample Number:32435
Sample Number:32436
Sample Number:32437
Sample Number:32438
Sample Number:32439
Sample Number:32440
Sample Number:32441
Sample Number:32442
Sample Number:32443
Sample Number:32444
Sample Number:32445
Sample Number:32446
Sample Number:32447
Sample Number:32448
Sample Number:32449
Sample Number:32450
Sample Number:32451
Sample Number:32452
Sample Number:32453
Sample Number:32454
Sample Number:32455
Sample Number:32456
Sample Number:32457
Sample Number:32458
Sample Number:32459
Sample Number:32460
Sample Number:32461
Sample Number:32462
Sample Number:32463
Sample Number:32464
Sample Number:32465


Sample Number:32830
Sample Number:32831
Sample Number:32832
Sample Number:32833
Sample Number:32834
Sample Number:32835
Sample Number:32836
Sample Number:32837
Sample Number:32838
Sample Number:32839
Sample Number:32840
Sample Number:32841
Sample Number:32842
Sample Number:32843
Sample Number:32844
Sample Number:32845
Sample Number:32846
Sample Number:32847
Sample Number:32848
Sample Number:32849
Sample Number:32850
Sample Number:32851
Sample Number:32852
Sample Number:32853
Sample Number:32854
Sample Number:32855
Sample Number:32856
Sample Number:32857
Sample Number:32858
Sample Number:32859
Sample Number:32860
Sample Number:32861
Sample Number:32862
Sample Number:32863
Sample Number:32864
Sample Number:32865
Sample Number:32866
Sample Number:32867
Sample Number:32868
Sample Number:32869
Sample Number:32870
Sample Number:32871
Sample Number:32872
Sample Number:32873
Sample Number:32874
Sample Number:32875
Sample Number:32876
Sample Number:32877
Sample Number:32878
Sample Number:32879


Sample Number:33245
Sample Number:33246
Sample Number:33247
Sample Number:33248
Sample Number:33249
Sample Number:33250
Sample Number:33251
Sample Number:33252
Sample Number:33253
Sample Number:33254
Sample Number:33255
Sample Number:33256
Sample Number:33257
Sample Number:33258
Sample Number:33259
Sample Number:33260
Sample Number:33261
Sample Number:33262
Sample Number:33263
Sample Number:33264
Sample Number:33265
Sample Number:33266
Sample Number:33267
Sample Number:33268
Sample Number:33269
Sample Number:33270
Sample Number:33271
Sample Number:33272
Sample Number:33273
Sample Number:33274
Sample Number:33275
Sample Number:33276
Sample Number:33277
Sample Number:33278
Sample Number:33279
Sample Number:33280
Sample Number:33281
Sample Number:33282
Sample Number:33283
Sample Number:33284
Sample Number:33285
Sample Number:33286
Sample Number:33287
Sample Number:33288
Sample Number:33289
Sample Number:33290
Sample Number:33291
Sample Number:33292
Sample Number:33293
Sample Number:33294


Sample Number:33663
Sample Number:33664
Sample Number:33665
Sample Number:33666
Sample Number:33667
Sample Number:33668
Sample Number:33669
Sample Number:33670
Sample Number:33671
Sample Number:33672
Sample Number:33673
Sample Number:33674
Sample Number:33675
Sample Number:33676
Sample Number:33677
Sample Number:33678
Sample Number:33679
Sample Number:33680
Sample Number:33681
Sample Number:33682
Sample Number:33683
Sample Number:33684
Sample Number:33685
Sample Number:33686
Sample Number:33687
Sample Number:33688
Sample Number:33689
Sample Number:33690
Sample Number:33691
Sample Number:33692
Sample Number:33693
Sample Number:33694
Sample Number:33695
Sample Number:33696
Sample Number:33697
Sample Number:33698
Sample Number:33699
Sample Number:33700
Sample Number:33701
Sample Number:33702
Sample Number:33703
Sample Number:33704
Sample Number:33705
Sample Number:33706
Sample Number:33707
Sample Number:33708
Sample Number:33709
Sample Number:33710
Sample Number:33711
Sample Number:33712


Sample Number:34074
Sample Number:34075
Sample Number:34076
Sample Number:34077
Sample Number:34078
Sample Number:34079
Sample Number:34080
Sample Number:34081
Sample Number:34082
Sample Number:34083
Sample Number:34084
Sample Number:34085
Sample Number:34086
Sample Number:34087
Sample Number:34088
Sample Number:34089
Sample Number:34090
Sample Number:34091
Sample Number:34092
Sample Number:34093
Sample Number:34094
Sample Number:34095
Sample Number:34096
Sample Number:34097
Sample Number:34098
Sample Number:34099
Sample Number:34100
Sample Number:34101
Sample Number:34102
Sample Number:34103
Sample Number:34104
Sample Number:34105
Sample Number:34106
Sample Number:34107
Sample Number:34108
Sample Number:34109
Sample Number:34110
Sample Number:34111
Sample Number:34112
Sample Number:34113
Sample Number:34114
Sample Number:34115
Sample Number:34116
Sample Number:34117
Sample Number:34118
Sample Number:34119
Sample Number:34120
Sample Number:34121
Sample Number:34122
Sample Number:34123


In [17]:
box_annotation

{'token': 'b0284e14d17a444a8d0071bd1f03a0a2',
 'sample_token': 'e93e98b63d3b40209056d129dc53ceee',
 'instance_token': 'a1f53dd65da64eb2b0cec51d57b8c047',
 'visibility_token': '4',
 'attribute_tokens': ['58aa28b1c2a54dc88e169808c07331e3'],
 'translation': [994.117, 639.415, 1.981],
 'size': [2.156, 6.227, 2.601],
 'rotation': [0.9503529315429128, 0.0, 0.0, -0.3111740758928862],
 'prev': '',
 'next': '405f156885d549ca812e1305eddee42f',
 'num_lidar_pts': 42,
 'num_radar_pts': 6,
 'category_name': 'vehicle.truck'}

In [3]:
def intersecting_points(points, boxes, ):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-3-c18552893530>, line 2)

In [ ]:
   #For each sample, loop over each sample_annotation
    for annotationToken in sample['anns']

annotation = nusc.get('sample_annotation',annotationToken)
        num_radar_points=annotation['num_radar_pts']
        
        #Make sure the annotation includes radar points, otherwise skip it
        if num_radar_points == 0:
            continue